To do:

- Parametrize gen_twisted_boundary over boundaries
- Encode the notions of "generator which transforms an
existing list of boundaries", "generator which transforms
another generator"
- Work directly with lists of boundaries. The only thing
I ever do with them is apply transforms to all of them, or
join adjacent ones with corresponding elements.
- Refactor/rewrite 'concat' with meshes because I think it
is scaling horribly. Perhaps have pre-allocated arrays that
scale by a factor of 1.5 every time they need to be grown, or
something.
- Why do I get the weird zig-zag pattern on the triangles,
despite larger numbers of them? Is it something in how I
twist the frames? (Note that I always connect one diagonal
on the boundaries - but I can choose the other. This might
not matter, but I should understand why.)

In [1]:
import stl.mesh
import numpy
import trimesh
import random

import meshutil
import examples

In [ ]:
#mesh = examples.ram_horn()
#mesh = examples.twist()
#mesh = examples.twist_nonlinear()
#mesh = examples.twist_from_gen()
#mesh = examples.twisty_torus(4000)

In [ ]:
# for me to be able to nest this, gen_twisted_boundary has to
# be parametrized over some other boundary (the way )
gen = examples.gen_inc_y(examples.gen_twisted_boundary())
mesh = gen2mesh(gen, 100, True)

In [ ]:
# Work around some annoying-ass trimesh/threejs bug:
wtf = meshutil.Transform().scale(0.1).translate(20,0,0)
center = meshutil.Transform().translate(-0.5, -0.5, -0.5)
base = meshutil.cube(open_xz=False).transform(center)
base = base.transform(wtf)
# to enable:
mesh_out = mesh.concat(base)

mesh_fname = "twist_wtf.stl"
mesh_out.to_stl_mesh().save(mesh_fname)

In [ ]:
# TODO: Just use the mesh data directly (no sense in saving & re-loading)
m = trimesh.load_mesh(mesh_fname)
#m.show()
scene = trimesh.Scene([m])
scene.show()